<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Basic-settings" data-toc-modified-id="Basic-settings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Basic settings</a></span></li><li><span><a href="#Child-Directed-Speech-~-fast-check" data-toc-modified-id="Child-Directed-Speech-~-fast-check-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Child Directed Speech ~ fast check</a></span><ul class="toc-item"><li><span><a href="#VSM" data-toc-modified-id="VSM-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>VSM</a></span></li><li><span><a href="#Clustering-*2" data-toc-modified-id="Clustering-*2-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Clustering *2</a></span></li><li><span><a href="#Grammar" data-toc-modified-id="Grammar-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Grammar</a></span></li></ul></li><li><span><a href="#Gutenberg-Children-Books-Caps" data-toc-modified-id="Gutenberg-Children-Books-Caps-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Gutenberg-Children-Books-Caps</a></span></li></ul></div>

# Agglomerative clustering in sparse vector space
Code implemented in development branch, waiting for merge after baseline tests.

## Basic settings

In [1]:
import os, sys, time
import numpy as np
import pandas as pd
from scipy import linalg
from IPython.display import display
from ull.grammar_learner.utl import UTC
from ull.grammar_learner.read_files import check_dir, check_mst_files
from ull.grammar_learner.pparser import files2links
from ull.grammar_learner.clustering import cluster_id 
from ull.grammar_learner.widgets import html_table
from ull.grammar_learner.write_files import list2file, save_link_grammar, save_cat_tree
from ull.grammar_learner.pqa_table import params, pqa_meter
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)
# Updated modules:
from src.grammar_learner.category_learner import \
    learn_categories, add_disjuncts, cats2list, cdf2cats
from src.grammar_learner.grammar_inducer import induce_grammar
# New (development) modules:
from src.grammar_learner_.sparse_word_space_ import \
    clean_links, co_occurrence_matrix, categorical_distribution
from src.grammar_learner_.agglomerative_clustering_ import agglomerative_clustering
start = time.time()
out_dir = module_path + '/output/Agglomerative_Clustering_' + str(UTC())[:10]
check_dir(out_dir, True, 'none')
tmpath = out_dir + '/tmp/'
check_dir(tmpath, True, 'none')
print(UTC(), '\n' + out_dir)
# cluster_id(26,26)  # Check (new) cluster numbering

2018-10-12 15:28:22 UTC 
/home/obaskov/py/language-learning/output/Agglomerative_Clustering_2018-10-12


In [2]:
kwargs = {
    'min_word_count':   1           ,   # SVS (sparse vector space) parameter
    'min_link_count':   1           ,   # SVS
    'max_words'     :   100000      ,   # SVS: max space dimension 1
    'max_features'  :   100000      ,   # SVS: dimension 2: disjuncts/connectors 
    'min_co-occurrence_count':  3   ,   # SVS
    'min_co-occurrence_probability': 1e-9,  # SVS
    'left_wall'     :   ''          ,
    'period'        :   False       ,
    'context'       :   1           ,
    'word_space'    :   'sparse'    ,
    'clustering'    :   ('agglomerative', 'ward'),      # new 
    'cluster_range' :   (30,120,3,3),   # min, max, step, repeat 
    'cluster_criteria': 'silhouette',   # legacy, to be replaced with:
    'clustering_metric': ('silhouette', 'euclidean'),   # new option (?)
    'cluster_level' :   1           ,
    'grammar_rules' :   2           ,
    'tmpath'        :   tmpath      , 
    'verbose'       :   'min'       ,
    'template_path' :   'poc-turtle',
    'linkage_limit' :   1000        ,
    'categories_generalization': 'off' }

## Child Directed Speech ~ fast check

### VSM

In [3]:
#%%time
corpus = 'CDS-caps-br-text+brent9mos'
#corpus = 'Gutenberg-Children-Books-Caps'
dataset = 'LG-English'
kwargs['min_word_count'] = 3   # 10 default 81010
kwargs['min_link_count'] = 1    # 1 default 81010
kwargs['min_co-occurrence_count'] = 1
#kwargs['min_co-occurrence_probability'] = 1e-3  # 0 default ~ categorical distribution threshold
ip, oc, og = params(corpus, dataset, module_path, out_dir, **kwargs)
rp = module_path + '/data/CDS-caps-br-text+brent9mos/LG-English'
#rp = ip  # reference path = Gutenberg-Children-Books-Caps/LG-English
cp = rp  # corpus path = reference_path :: use 'gold' parses as test corpus
files, re01 = check_mst_files(ip, verbose='none')
kwargs['input_files'] = files
links, re02 = files2links(**kwargs)
linx, words, features = clean_links(links, **kwargs)

clean_links: min_word_count: 3, min_link_count: 1


In [4]:
print(f'{len(links)} links: {len(set(links["word"].tolist()))} unique words, {len(set(links["link"].tolist()))} links')
print(f'words: len {len(words)}, min {min(words)}, max {max(words)}')
print(f'features: len {len(features)}, min {min(features)}, max {max(features)}')
l0 = np.unique(linx[:,0])
l1 = np.unique(linx[:,1])
print(f'linx[0]: len {len(l0)}, min {min(l0)}, max {max(l0)}')
print(f'linx[1]: len {len(l1)}, min {min(l1)}, max {max(l1)}')
counts = co_occurrence_matrix(linx, **kwargs)
cd = categorical_distribution(counts, **kwargs)
print(f'counts.shape {counts.shape},cd.shape {cd.shape}')

49570 links: 3306 unique words, 5285 links
words: len 2222, min 'd, max zoo
features: len 3981, min 'd+, max zoo-
linx[0]: len 2222, min 0, max 2221
linx[1]: len 3965, min 0, max 3980
co_occurrence_matrix: threshold = 1
categorical_distribution: threshold = 0.001
counts.shape (2222, 3981),cd.shape (2222, 3981)


### Clustering *2

In [5]:
def cluster_sizes(cdf):
    cs = {}
    for x in cdf['cluster_words'].tolist():
        if len(x) in cs: cs[len(x)] += 1
        else: cs[len(x)] = 1
    return sorted(cs, reverse=True)

In [6]:
kwargs['clustering'] = ('agglomerative', 'ward')
for n_clusters in range(5,10):
    cdf, silhouette, m2, labels = agglomerative_clustering(cd, words, n_clusters, **kwargs)
    print(f'{kwargs["clustering"][0]}: {n_clusters} clusters {cluster_sizes(cdf)}, SI {round(silhouette, 3)}, VR {int(m2)}')

agglomerative: 5 clusters [2204, 8, 1], SI 0.983, VR 658
agglomerative: 6 clusters [2201, 8, 3, 1], SI 0.985, VR 704
agglomerative: 7 clusters [2201, 8, 7, 3, 1], SI 0.985, VR 704
agglomerative: 8 clusters [2201, 8, 5, 3, 2, 1], SI 0.986, VR 693
agglomerative: 9 clusters [2200, 8, 5, 3, 2, 1], SI 0.986, VR 700


In [7]:
cdf

,cluster,cluster_words
0,B,"[the, to, a, and, get, in, put, i, see, we, ha..."
1,C,"[that, it, there, he, where]"
2,D,"[is, do, are]"
3,E,"[on, here]"
4,F,['s]
5,G,"[go, can, 're, want, gonna, don't, wanna, can't]"
6,H,[what]
7,I,[you]
8,J,[no]


In [8]:
for n_clusters in [5,10,20,21,22,23,24,25,30]:
    cdf, silhouette, m2, labels = agglomerative_clustering(cd, words, n_clusters, **kwargs)
    print(f'{kwargs["clustering"][0]}: {n_clusters} clusters {cluster_sizes(cdf)}, SI {round(silhouette, 3)}, VR {int(m2)}')

agglomerative: 5 clusters [2204, 8, 1], SI 0.983, VR 658
agglomerative: 10 clusters [2199, 8, 5, 3, 2, 1], SI 0.987, VR 726
agglomerative: 20 clusters [2192, 8, 3, 2, 1], SI 0.99, VR 1736
agglomerative: 21 clusters [2191, 8, 3, 2, 1], SI 0.99, VR 2211
agglomerative: 22 clusters [2191, 7, 3, 2, 1], SI 0.99, VR 3013
agglomerative: 23 clusters [2191, 6, 3, 2, 1], SI 0.99, VR 4841
agglomerative: 24 clusters [2191, 5, 3, 2, 1], SI 0.991, VR 12502
agglomerative: 25 clusters [2191, 5, 3, 2, 1], SI 0.991, VR 1
agglomerative: 30 clusters [2191, 2, 1], SI nan, VR 1


/home/obaskov/miniconda3/envs/ull_dev/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


### Grammar

In [9]:
n_clusters = 7
cdf, silhouette, m2, labels = agglomerative_clustering(cd, words, n_clusters, **kwargs)
print(f'{kwargs["clustering"][0]}: {n_clusters} clusters {cluster_sizes(cdf)},\
 SI {round(silhouette, 3)}, VR = {int(m2)}')

agglomerative: 7 clusters [2201, 8, 7, 3, 1], SI 0.985, VR = 704


In [10]:
if kwargs['context'] == 1 and kwargs['grammar_rules'] == 2:
    context = kwargs['context']
    kwargs['context'] = kwargs['grammar_rules']
    links, re06 = files2links(**kwargs)
    kwargs['context'] = context

In [11]:
cats = cdf2cats(cdf)
gen_cats = cats
fat_cats = add_disjuncts(gen_cats, links, verbose='none')
rules, re07 = induce_grammar(fat_cats, links)
re09 = save_cat_tree(rules,  oc, verbose='none')
grammar_rules = 2
re10 = save_link_grammar(rules, og, grammar_rules)

In [12]:
%%capture
pa, pq, qa = pqa_meter(re10['grammar_file'], og, cp, rp, **kwargs)

In [13]:
print(f'PA/PQ = {round(pa,1)}%/{round(pq,1)}%')

PA/PQ = 70.9%/49.8%


In [14]:
#STOP

## Gutenberg-Children-Books-Caps

In [ ]:
def gogo(n_clusters, ip, oc, og, cp, rp, **kwargs):
    files, re01 = check_mst_files(ip, verbose='none')
    kwargs['input_files'] = files
    links, re02 = files2links(**kwargs)
    linx, words, features = clean_links(links, **kwargs)
    counts = co_occurrence_matrix(linx, **kwargs)
    cd = categorical_distribution(counts, **kwargs)
    print(f'counts.shape: {counts.shape}, cd.shape: {cd.shape}')
    print(f'{len(links)} links: {len(set(links["word"].tolist()))} unique words, {len(set(links["link"].tolist()))} links')
    print(f'words: len {len(words)}, min {min(words)}, max {max(words)}')
    print(f'features: len {len(features)}, min {min(features)}, max {max(features)}')
    l0 = np.unique(linx[:,0])
    l1 = np.unique(linx[:,1])
    print(f'linx[0]: len {len(l0)}, min {min(l0)}, max {max(l0)}')
    print(f'linx[1]: len {len(l1)}, min {min(l1)}, max {max(l1)}')
    cdf, silhouette, m2, labels = agglomerative_clustering(cd, words, n_clusters, **kwargs)
    print(f'{kwargs["clustering"][0]}: {n_clusters} clusters, SI {round(silhouette, 3)}, VR = {int(m2)}')
    print('cluster sizes:', cluster_sizes(cdf))
    cats = cdf2cats(cdf)
    gen_cats = cats
    if kwargs['context'] == 1 and kwargs['grammar_rules'] == 2:
        context = kwargs['context']
        kwargs['context'] = kwargs['grammar_rules']
        links, re06 = files2links(**kwargs)
        kwargs['context'] = context
    fat_cats = add_disjuncts(gen_cats, links, verbose='none')
    rules, re07 = induce_grammar(fat_cats, links)
    re09 = save_cat_tree(rules,  oc, verbose='none')
    grammar_rules = 2
    re10 = save_link_grammar(rules, og, kwargs['grammar_rules'])
    pa, pq, qa = pqa_meter(re10['grammar_file'], og, cp, rp, **kwargs)
    return pa, pq, cdf

In [ ]:
corpus = 'Gutenberg-Children-Books-Caps'
dataset = 'LG-English'
ip, oc, og = params(corpus, dataset, module_path, out_dir, **kwargs)
rp = ip  # reference path = LG-English
cp = rp  # corpus path = reference path
kwargs['min_word_count'] = 31
kwargs['min_link_count'] = 2
kwargs['min_co-occurrence_count'] = 1
kwargs['min_co-occurrence_probability'] = 1e-9  # categorical distribution threshold
kwargs['clustering'] = ('agglomerative', 'ward') # linkage: ('ward', 'average', 'complete')

In [ ]:
%%capture
n_clusters = 100
pa, pq, cdf = gogo(n_clusters, ip, oc, og, cp, rp, **kwargs)

In [ ]:
print(f'PA/PQ = {round(pa,1)}%/{round(pq,1)}%')
print('cluster sizes:', cluster_sizes(cdf))

1st take: **1-connector-disjunct rules**:  
PA/PQ = 14.1%/5.5%  
cluster sizes: [1703, 1085, 875, 732, 703, 276, 269, 202, 172, 156, 111, 109, 83, 72, 51, 37, 26, 21, 19, 17, 11, 10, 8, 7, 5, 4, 3, 2, 1]